# Pendahuluan
Pada saat melakukan analisis, kita perlu merumuskan beberapa hipotesis yang perlu kita uji lebih terlebih dahulu. Tidak semua hipotesis yang kita kerjakan bisa langsung digunakan, kita harus bisa memilih hipotesis mana yang paling tepat untuk menyelesaikan masalah-masalah yang ada pada data yang kita miliki. Untuk membuat keputusan yang tepat, kita harus memahami apakah asumsi yang kita buat sudah tepat atau belum.

# Deskripsi Proyek
Rusty Bargain adalah perusahaan jual beli mobil bekas yang sedang mengembangkan sebuah aplikasi untuk menarik pembeli baru. Dalam aplikasi tersebut, kamu bisa dengan cepat mengetahui nilai pasar mobilmu. Kamu memiliki akses ke data historis, spesifikasi teknis kendaraan, versi model kendaraan, dan harga kendaraan. Tugasmu adalah membuat suatu model yang mampu menentukan nilai pasar mobil.

Rusty Bargain tertarik pada:
- kualitas prediksi;
- kecepatan model dalam memprediksi;
- waktu yang diperlukan untuk melatih model

# Tujuan
Membuat suatu model yang mampu menentukan nilai pasar mobil.

# Instruksi Proyek
1. Simpan dan amati datanya.
2. Latih model yang berbeda dengan berbagai hyperparameter (Kamu harus membuat setidaknya dua model yang berbeda, tetapi lebih banyak lebih baik. Ingat, berbagai implementasi dari gradient boosting tidak dihitung sebagai model yang berbeda). Poin utama dari langkah ini adalah untuk membandingkan metode-metode gradient boosting dengan random forest, decision tree, dan regresi linear.
3. Analisis kecepatan dan kualitas model.

Catatan:
- Gunakan metrik RMSE untuk mengevaluasi model.
- Regresi linear memang tidak terlalu cocok untuk mengatur hyperparameter, tetapi regresi linear merupakan model yang tepat untuk melakukan sanity check dibandingkan metode lainnya. Jika kinerja gradient boosting tidak lebih baik daripada regresi linear, berarti ada sesuatu yang salah.
- Secara mandiri, pelajari library LightGBM dan gunakan tool yang ada di dalam library tersebut untuk membuat model gradient boosting.
- Idealnya, tugasmu harus menyertakan regresi linear untuk melakukan sanity check, algoritma berbasis pohon dengan penyetelan hyperparameter (sebaiknya random forest), LightGBM dengan penyetelan hyperparameter (coba beberapa set), serta CatBoost dan XGBoost dengan penyetelan hyperparameter (opsional).
- Perhatikan pengodean terhadap fitur-fitur kategorik untuk algoritma sederhana. LightGBM dan CatBoost memang mengimplementasikannya, tetapi XGBoost membutuhkan OHE.
- Kamu bisa menggunakan perintah khusus untuk mengetahui waktu pemrosesan suatu sel kode di Jupyter Notebook. Temukan perintah khusus tersebut.
- Mengingat pelatihan model gradient boosting bisa membutuhkan waktu yang lama, ubah hanya beberapa parameter model saja.
- Jika Jupyter Notebook berhenti bekerja, hapus variabel yang tidak diperlukan menggunakan operator del: 
    
    **del features_train**

# Deskripsi Data
Dataset yang kamu butuhkan tersimpan di file/datasets/car_data.csv. Unduh dataset.

**Fitur :**
- DateCrawled — tanggal ketika profil diunduh dari database
- VehicleType — jenis bodi kendaraan
- RegistrationYear — tahun registrasi kendaraan
- Gearbox — jenis transmisi
- Power — daya (hp)
- Model — model kendaraan
- Mileage — jarak tempuh (diukur dalam satuan km berdasarkan dataset regional tertentu)
- RegistrationMonth — bulan registrasi kendaraan
- FuelType — jenis bahan bakar
- Brand — merek kendaraan
- NotRepaired — apakah kendaraan pernah direparasi sebelumnya
- DateCreated — tanggal pembuatan profil
- NumberOfPictures — jumlah gambar kendaraan
- PostalCode — kode pos pemilik profil (pengguna)
- LastSeen — tanggal aktivitas terakhir pengguna

**Target :**
- Price — harga (dalam satuan Euro)

## Persiapan Data

In [1]:
# Import Library
import numpy as np
import pandas as pd

import seaborn as sns

import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

### Muat Data
Muat data dan lakukan pemeriksaan untuk memastikan data bebas dari permasalahan.

In [2]:
data = pd.read_csv('/datasets/car_data.csv')

In [3]:
# Menampilkan data
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,21/03/2016 09:50,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,21/03/2016 00:00,0,2694,21/03/2016 10:42
354365,14/03/2016 17:48,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46
354366,05/03/2016 19:56,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,05/03/2016 00:00,0,26135,11/03/2016 18:17
354367,19/03/2016 18:57,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,19/03/2016 00:00,0,87439,07/04/2016 07:15


In [4]:
# Informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# Menampilkan deskriptif statitik data 
data.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
# Mencari nilai yang hilang 
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

**Catatan:**

Terdapat data yang hilang pada kolom VehicleType, Gearbox, Model, FuelType, dan NotRepaired

In [7]:
# Menangani nilai yang hilang
data['VehicleType'].fillna('unknown', inplace=True)
data['Gearbox'].fillna('unknown', inplace=True)
data['Model'].fillna('unknown', inplace=True)
data['FuelType'].fillna('unknown', inplace=True)
data['NotRepaired'].fillna('unknown', inplace=True)


In [8]:
# Check apakah masih ada nilai yang hilang
data.isnull().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [9]:
# Mengonversi tipe data
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%d/%m/%Y %H:%M')
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%d/%m/%Y %H:%M')
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%d/%m/%Y %H:%M')

**Kesimpulan Tahap 1:**
- Terdapat data yang hilang pada kolom VehicleType, Gearbox, Model, FuelType, dan NotRepaired dan sudah dilakukan penanganan terhadap nilai yang hilang tersebut. 
- Dilakukan konversi tipe data pada data di kolom DateCrawled, DateCreated, dan LastSeen.

## Pelatihan Model

### Preprocessing Data / Pra-pemrosessan data

In [10]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:00,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,2016-03-24,0,70435,2016-04-07 03:16:00
1,2016-03-24 10:58:00,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:00
2,2016-03-14 12:52:00,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,2016-03-14,0,90480,2016-04-05 12:47:00
3,2016-03-17 16:54:00,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:00
4,2016-03-31 17:25:00,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-04-06 10:17:00


In [11]:
# Menangani fitur kategorikal
data = pd.get_dummies(data, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first=True)

In [12]:
# Menentukan variabel target (y) dan fitur (X)
X = data.drop(['Price', 'DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
y = data['Price']

### Pembagian Data

In [13]:
# Membagi data menjadi data set pelatihan dan pengujian.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

### Latih Model

#### Regresi Linear

In [14]:
linear_model = sklearn.linear_model.LinearRegression()
linear_model.fit(X_train, y_train)
linear_predictions = linear_model.predict(X_test)
linear_rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, linear_predictions))
print('Linear Regression RMSE:', linear_rmse)

Linear Regression RMSE: 3178.4867053025187


#### Random Forest

In [15]:
random_forest_model = RandomForestRegressor(random_state=12)
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_test)
random_forest_rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, random_forest_predictions))
print("Random Forest RMSE:", random_forest_rmse)

Random Forest RMSE: 1698.2899594433409


#### LightGBM

In [16]:
lgb_model = lgb.LGBMRegressor(random_state=12)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, lgb_predictions))
print("LightGBM RMSE:", lgb_rmse)

LightGBM RMSE: 1836.9102377706436


####  CatBoost

In [17]:
catboost_model = CatBoostRegressor(random_state=12, silent=True)
catboost_model.fit(X_train, y_train)
cat_predictions = catboost_model.predict(X_test)
cat_rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, cat_predictions))
print("CatBoost RMSE:", cat_rmse)

CatBoost RMSE: 1748.971695781754


#### XGBoost 

In [18]:
xgb_model = xgb.XGBRegressor(random_state=12)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, xgb_predictions))
print("XGBoost RMSE:", xgb_rmse)

XGBoost RMSE: 1801.5959516878413


**Kesimpulan Tahap 2:**

Hasil RMSE untuk model-model yang sudah dilatih :
- Linear Regression RMSE: 3178.4867053025187
- Random Forest RMSE: 1698.2899594433409
- LightGBM RMSE: 1836.9102377706436
- CatBoost RMSE: 1748.971695781754
- XGBoost RMSE: 1801.5959516878413

## Analisis Model
Setelah melatih model, kita bisa menganalisis kualitas dan kecepatan model berdasarkan nilai RMSE dan waktu pelatihan.

In [19]:
# Mengevaluasi kecepatan dan kualitas model
model_names = ['Linear Regression', 'Random Forest', 'LightGBM', 'CatBoost', 'XGBoost']
rmse_values = [linear_rmse, random_forest_rmse, lgb_rmse, cat_rmse, xgb_rmse]

for name, rmse in zip(model_names, rmse_values):
    print(f"{name} RMSE: {rmse}")

Linear Regression RMSE: 3178.4867053025187
Random Forest RMSE: 1698.2899594433409
LightGBM RMSE: 1836.9102377706436
CatBoost RMSE: 1748.971695781754
XGBoost RMSE: 1801.5959516878413


**Kesimpulan Tahap 3:**
- Random Forest memiliki RMSE terendah diantara semua model yang diuji, hal ini menunjukkan kinerja yang lebih baik dari model yang lainnya. 

## Kesimpulan Umum
**Kesimpulan Tahap 1:**
- Terdapat data yang hilang pada kolom VehicleType, Gearbox, Model, FuelType, dan NotRepaired dan sudah dilakukan penanganan terhadap nilai yang hilang tersebut.
- Dilakukan konversi tipe data pada data di kolom DateCrawled, DateCreated, dan LastSeen.

**Kesimpulan Tahap 2:**

Hasil RMSE untuk model-model yang sudah dilatih :
- Linear Regression RMSE: 3178.4867053025187
- Random Forest RMSE: 1698.2899594433409
- LightGBM RMSE: 1836.9102377706436
- CatBoost RMSE: 1748.971695781754
- XGBoost RMSE: 1801.5959516878413

**Kesimpulan Tahap 3:**
- Random Forest memiliki RMSE terendah diantara semua model yang diuji, hal ini menunjukkan kinerja yang lebih baik dari model yang lainnya. 